## Preprocessing

In [1]:
# Import dependencies
from sklearn.model_selection import KFold, cross_val_score 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from scikeras.wrappers import KerasClassifier 

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
def application_types_to_replace(df, column_name, cutoff=500): # Threshold for rarity
    frequency = df[column_name].value_counts()
    categories_to_replace = [category for category, count in frequency.items() if count < cutoff]
    df[column_name] = df[column_name].replace(categories_to_replace, 'Other')
    return df

# Replace rare values in 'APPLICATION_TYPE'
application_df = application_types_to_replace(application_df, 'APPLICATION_TYPE')

# Check if the replacement was successful by viewing the updated frequency distribution
print(application_df['APPLICATION_TYPE'].value_counts())


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
filtered_counts = classification_counts[classification_counts > 1]
print(filtered_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
def classifications_to_replace(df, column_name, cutoff=100): # Threshold for rarity
    frequency = df[column_name].value_counts()
    categories_to_replace = [category for category, count in frequency.items() if count < cutoff]
    df[column_name] = df[column_name].replace(categories_to_replace, 'Other')
    return df

# Replace rare values in 'CLASSIFICATION' columns
application_df = classifications_to_replace(application_df, 'CLASSIFICATION')
print(application_df['CLASSIFICATION'].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64


In [9]:
# Define the bin ranges and corresponding labels
bins = [0, 50000, 100000, 150000, 250000, 500000, 750000, 1000000, float('inf')]
labels = ['0-50K', '50K-100K', '100K-150K', '150K-250K', '250K-500K', '500K-750K', '750K-1M', '1M+']

# Apply the binning to the 'ASK_AMT' column
application_df['ASK_AMT_binned'] = pd.cut(application_df['ASK_AMT'], bins=bins, labels=labels)


# Drop the original 'ASK_AMT' column as it's now binned
application_df.drop(columns=['ASK_AMT'], inplace=True)

# Show the updated dataframe with the binned column
application_df.head()

APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0              T10       Independent          C1000    ProductDev   
1               T3       Independent          C2000  Preservation   
2               T5  CompanySponsored          C3000    ProductDev   
3               T3  CompanySponsored          C2000  Preservation   
4               T3       Independent          C1000     Heathcare   

   ORGANIZATION  STATUS     INCOME_AMT SPECIAL_CONSIDERATIONS  IS_SUCCESSFUL  \
0   Association       1              0                      N              1   
1  Co-operative       1         1-9999                      N              1   
2   Association       1              0                      N              0   
3         Trust       1    10000-24999                      N              1   
4         Trust       1  100000-499999                      N              1   

  ASK_AMT_binned  
0          0-50K  
1      100K-150K  
2          0-50K  
3          0-50K  
4      100K-150K

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df, columns=['ASK_AMT_binned', 'INCOME_AMT', 'APPLICATION_TYPE', 
                                                                 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 
                                                                 'ORGANIZATION', 'STATUS', 'SPECIAL_CONSIDERATIONS'])
print(application_df_encoded)

       IS_SUCCESSFUL  ASK_AMT_binned_0-50K  ASK_AMT_binned_50K-100K  \
0                  1                  True                    False   
1                  1                 False                    False   
2                  0                  True                    False   
3                  1                  True                    False   
4                  1                 False                    False   
...              ...                   ...                      ...   
34294              0                  True                    False   
34295              0                  True                    False   
34296              0                  True                    False   
34297              1                  True                    False   
34298              0                 False                    False   

       ASK_AMT_binned_100K-150K  ASK_AMT_binned_150K-250K  \
0                         False                     False   
1                        

In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df_encoded['IS_SUCCESSFUL']
X = application_df_encoded.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_train_scaled = scaler.fit_transform(X_train)

# Scale the data
X_test_scaled = scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Define the neural network model creation function
def create_nn_model_optimized(input_dim):
    model = tf.keras.models.Sequential()
    
    # Input layer
    model.add(layers.Input(shape=(input_dim,)))  
    
    # Hidden layers
    model.add(layers.Dense(units=512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))  
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))  
    
    model.add(layers.Dense(units=256, activation='relu', kernel_regularizer=regularizers.l2(0.001)))  
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))  
    
    model.add(layers.Dense(units=128, activation=tf.keras.layers.LeakyReLU(negative_slope=0.01), kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))  
    
    # Output layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Binary classification

    # Compile the model
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001, decay_steps=100, decay_rate=0.96, staircase=True)
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics=['accuracy'])

     # Print model summary for debugging
    model.summary()
    
    return model


In [ ]:
# Wrap the Keras model into a scikit-learn compatible classifier
model = KerasClassifier(build_fn=create_nn_model_optimized, input_dim=X_train_scaled.shape[1], epochs=10, batch_size=32, verbose=1)  # Set verbose=1 for debugging

# Create a KFold cross-validation object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation using the KFold object
cv_scores = cross_val_score(
    estimator=model,  # Use the wrapped Keras model
    X=X_train_scaled,  # Features
    y=y_train,  # Target variable
    cv=kf,  # Use KFold
    scoring='accuracy',  # Evaluation metric (accuracy in this case)
    n_jobs=1  # Set n_jobs=1 to avoid parallelization issues
)

# Print cross-validation results
print(f"Cross-validation accuracy scores: {cv_scores}")
print(f"Average accuracy: {cv_scores.mean()} ± {cv_scores.std()}")



In [ ]:
# After cross-validation, train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model manually with early stopping
nn_model_optimized = create_nn_model_optimized(X_train_scaled.shape[1])  # Create the model again
history = nn_model_optimized.fit(X_train_scaled, y_train, epochs=200, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_optimized.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7308 - loss: 0.6010
Loss: 0.6009949445724487, Accuracy: 0.7308454513549805


In [18]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity_Optimization.h5')

NameError: name 'nn_model' is not defined

In [ ]:
# Visualize training history (if needed)
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()